In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

import json
import os

# Specify the path to your Kaggle API credentials JSON file
kaggle_json_path = '/content/kaggle(2).json'  # Update with the actual path

# Check if the JSON file exists
if os.path.exists(kaggle_json_path):
    with open(kaggle_json_path, 'r') as json_file:
        kaggle_credentials = json.load(json_file)

    # Extract username and key from the JSON file
    kaggle_username = kaggle_credentials['username']
    kaggle_key = kaggle_credentials['key']

    # Set Kaggle API credentials as environment variables
    os.environ['KAGGLE_USERNAME'] = kaggle_username
    os.environ['KAGGLE_KEY'] = kaggle_key
else:
    print("Kaggle API credentials JSON file not found.")


import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!kaggle datasets download -d mahibuzzaman/multicancer-all-classes7-cancer-23classes224x224 -p '/content/'


import zipfile

with zipfile.ZipFile('/content/multicancer-all-classes7-cancer-23classes224x224.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

KeyboardInterrupt: ignored

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import random





import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout,AveragePooling2D
from tensorflow.keras.models import Model


from tensorflow.keras import Sequential

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import BatchNormalization, Input, Conv2D, MaxPooling2D, Flatten, Dense

# Define the common convolutional layers
from tensorflow.keras.layers import BatchNormalization, Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Assuming 'main_class' and 'sub_class' are string columns, convert them to a categorical type
train_df['main_class'] = train_df['main_class'].astype('category')
train_df['sub_class'] = train_df['sub_class'].astype('category')

val_df['main_class'] = val_df['main_class'].astype('category')
val_df['sub_class'] = val_df['sub_class'].astype('category')

test_df['main_class'] = test_df['main_class'].astype('category')
test_df['sub_class'] = test_df['sub_class'].astype('category')

IMG_SIZE = 200

# Load the pre-trained VGG16 model with weights trained on ImageNet data
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a new input layer for your custom classification layers
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Connect your input to the base VGG16 model
x = base_model(inputs)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)

# Main class classification
main_class_output = Dense(train_df['main_class'].nunique(), activation='softmax', name='output_main')(x)

# Create a model for main class classification
model_main = Model(inputs=inputs, outputs=[main_class_output])

model_main.compile(optimizer=Adam(lr=0.0001),
                   loss={'output_main': 'categorical_crossentropy'},
                   metrics={'output_main': 'accuracy'})

# Create data generator for main_class prediction
datagen_main = ImageDataGenerator(rescale=1.0 / 255.0)

output_block1_batch_size = 512

output_block1_train_generator = datagen_main.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='main_class',
    class_mode='categorical',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=output_block1_batch_size
)

output_block1_val_generator = datagen_main.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='main_class',
    class_mode='categorical',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=output_block1_batch_size
)

# Train the main_class prediction model
model_main.fit(
    output_block1_train_generator,
    epochs=10,  # Adjust the number of epochs
    validation_data=output_block1_val_generator
)

# Predict main classes for the test dataset
output_block1_test_generator = datagen_main.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='main_class',
    class_mode='categorical',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=output_block1_batch_size,
    shuffle=False  # Important: Do not shuffle for evaluation
)

main_class_predictions = model_main.predict(output_block1_test_generator)
predicted_main_classes = np.argmax(main_class_predictions, axis=1)

# Preprocess images based on predicted main class
for idx, main_class in enumerate(main_classes):
    # Filter data for the current main_class
    sub_block2_test = test_df[test_df['main_class'] == main_class]

    # Preprocess images here based on the predicted main class (e.g., resize, normalization)

    # Create a new input layer for your custom sub_class classification layers
    inputs_sub = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    # Connect your input to the base VGG16 model
    x_sub = base_model(inputs_sub)
    x_sub = Flatten()(x_sub)
    x_sub = Dense(256, activation='relu')(x_sub)

    # Sub class classification
    sub_class_output = Dense(sub_block2_test['sub_class'].nunique(), activation='softmax', name='output_sub')(x_sub)

    model_sub = Model(inputs=inputs_sub, outputs=[sub_class_output])

    model_sub.compile(optimizer=Adam(lr=0.0001),
                      loss={'output_sub': 'categorical_crossentropy'},
                      metrics={'output_sub': 'accuracy'})

    # Create data generator for sub_class prediction
    datagen_sub = ImageDataGenerator(rescale=1.0 / 255.0)

    output_block2_batch_size = 512

    output_block2_test_generator = datagen_sub.flow_from_dataframe(
        dataframe=sub_block2_test,
        x_col='image_path',
        y_col='sub_class',
        class_mode='categorical',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=output_block2_batch_size,
        shuffle=False  # Important: Do not shuffle for evaluation
    )

    # Train the sub_class prediction model for the current main_class
    model_sub.fit(
        output_block2_test_generator,
        epochs=10,  # Adjust the number of epochs
        validation_data=output_block2_val_generator
    )

    sub_models[main_class] = model_sub

# Now, you can use sub_models dictionary to make subclass predictions for each main class

# Evaluate on the test dataset
main_class_eval = model_main.evaluate(output_block1_test_generator)

print("Main Class Evaluation - Loss:", main_class_eval[0], "Accuracy:", main_class_eval[1])

subclass_evaluations = {}

for main_class in main_classes:
    # Filter data for the current main_class
    sub_block2_test = test_df[test_df['main_class'] == main_class]

    output_block2_test_generator = datagen_sub.flow_from_dataframe(
        dataframe=sub_block2_test,
        x_col='image_path',
        y_col='sub_class',
        class_mode='categorical',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=output_block2_batch_size,
        shuffle=False  # Important: Do not shuffle for evaluation
    )

    subclass_eval = sub_models[main_class].evaluate(output_block2_test_generator)
    subclass_evaluations[main_class] = subclass_eval

    print(f"Subclass Evaluation for Main Class {main_class} - Loss:", subclass_eval[0], "Accuracy:", subclass_eval[1])
